# Course1-Project1: EdTech Backend System

In [ ]:
import hashlib
from openpyxl import load_workbook

file_path = r"C:\Users\ecarald\OneDrive - Ericsson\Desktop\PG DE\Course 1\instructors.xlsx"

def set_id(name):
    # Encode the string and compute a hash (SHA-256)
    hash_name = hashlib.sha256(name.encode('utf-8'))
    # Convert the hexadecimal hash to an integer
    id = int(hash_name.hexdigest(), 16)
    # Take only part of it (last 10 digits)
    return id % (10**10)
        
class User:
    counter = 0
    
    def __init__(self, name, role, email=None, password=None):
        '''
        'name' and 'role' are required at object creation.
        'role' can be only either 'Learner' or 'Instructor'
        'email' and 'password' are optional.
        '''  
        self.name = name
        self.id=set_id(name)
        self.role = role
        self.email = email
 
        if password:
            self.hash_password(password)
            self.dotify_password(password)

        User.counter +=1

    def hash_password(self, password):
        '''
        Hash the password using SHA-256 before storing it
        '''
        # encode to bytes and hash with sha256
        hashed = hashlib.sha256(password.encode("utf-8")).hexdigest()
        self.__password_hash = hashed
        
    def dotify_password(self, password):
        '''
        Mask the password with dots
        '''
        self.dotified_password = '.' * len(password)

    def validate_password(self,password):
        """
        Check if a given password matches the stored hashed password.
        """
        if self.__password_hash is None:
            return False
        hashed = hashlib.sha256(password.encode("utf-8")).hexdigest()
        return hashed == self.__password_hash       

class Instructor(User):
    def __init__(self, name, email):
        User.__init__(self, name, "Instructor", email)
        self.taught_courses = [] #list of course names

    def add_course(self, course_name):
        '''Adds a course to the instructor's taught_courses list if not already present.'''
        if course_name not in self.taught_courses:
            self.taught_courses.append(course_name)
            print(f"Course '{course_name}' added to {self.name}.")
        else:
            print(f"{self.name} already teaches '{course_name}'.")

    def remove_course(self, course_name):
        '''Removes course from the instructor´s taught_courses list if it exists.'''
        if course_name in self.taught_courses:
            self.taught_courses.remove(course_name)
            print(f"Course {course_name} removed from {self.name}.")
        else:
            print(f"{self.name} does not teaches '{course_name}'.")
    
    def __str__(self):
        return f"Instructor name: {self.name}\nInstructor ID: {self.id}\nTaught courses: {self.taught_courses}"

class Learner(User):
    def __init__(self, name, email):
        User.__init__(self, name, "Learner", email)
        self.enrolled_courses = [] #list of course names

    def enroll_course(self, course_name):
        if course_name not in self.enrolled_courses:
            self.enrolled_courses.append(course_name)
            print(f"Course '{course_name}' added to {self.name}.")
        else:
            print(f"{self.name} is already enrolled in '{course_name}'.")

    def drop_course(self, course_name):
        if course_name in self.enrolled_courses:
            self.enrolled_courses.remove(course_name)
            print(f"Course {course_name} removed from {self.name}.")
        else:
            print(f"{self.name} is not enrolled in '{course_name}'.")

class Course:
    def __init__(self, name):
        self.name = name
        self.id = set_id(name)
        self.enrolled_learners = [] # list of enrolled learners

    def add_learner(self, learner_name):
        '''Adds a learner to the courses´s enrolled learners list if not already present.'''
        if learner_name not in self.enrolled_learners:
            self.enrolled_learners.append(course_name)
            print(f"Learner '{learner_name}' successfully added to {self.name}.")
        else:
            print(f"{learner_name} is already enrolled in '{self.name}'.")
            
    def remove_learner(self, learner_name):
        '''Removes learner from the courses´s enrolled learners list if it exists.'''
        if learner_name in self.enrolled_learners:
            self.enrolled_learners.remove(learner_name)
            print(f"Learner '{learner_name}' successfully removed from {self.name}.")
        else:
            print(f"{learner_name} not found in '{self.name}'.")        
    
    def __str__(self):        
        return f"Course name: {self.name}\nCourse ID: {self.id}"
        

class Enrollment:    
    def __init__(self, learner, course):
        self.learner = learner         # instance of Learner
        self.course = course           # instance of Course
        self.status = "active"         # or "completed", "dropped", etc.

class Backend(): # acts as the sort of database manager
    def __init__(self): 
        self.instructors = [] #'instructors' is a simple list of instances of class Instructor
        self.learners = []
        self.courses = []
        self.enrollments = []

    def load_instructors_from_excel(self, file_path):           
        # Load workbook and select first sheet
        workbook = load_workbook(filename=file_path)
        sheet = workbook.active        

        # Skip the header row → start from row 2
        for row in sheet.iter_rows(min_row=2, values_only=True):
            name = row[0]
            surname = row[1]
            email = row[2]
            
            full_name = f"{name} {surname}"
            instructor = Instructor(full_name, email)

            for course_name in row[3:]:
                if course_name is not None: # skip empty cells
                    instructor.add_course(course_name)
            
            self.instructors.append(instructor)

    def _add_object_to_list(self, new_object, list_name, list_label):
        '''Private helper method to add one object element to a given list'''
        existing = next((obj for obj in list_name if obj.id == new_object.id), None)
        if not existing:
            list_name.append(new_object)
            print(f"'{new_object.name}' successfully added to the {list_label} list.")
        else:
            print(f"'{new_object.name}' already exists in {list_label} list.")

    def add_instructor(self, instructor):
        self._add_object_to_list(instructor, self.instructors, "Instructors")
    def add_learner(self, learner):
        self._add_object_to_list(learner, self.learners, "Learners")        
    def add_course(self, course):
        self._add_object_to_list(course, self.courses, "Courses")
        
    def _remove_object_from_list(self, object_to_remove, list_name, list_label):
        '''Private helper method to remove one object element from a given list'''
        existing = next((obj for obj in list_name if obj.id == object_to_remove.id), None)
        if existing:
            list_name.remove(object_to_remove)
            print(f"'{object_to_remove.name}' successfully removed from the {list_label} list.")
        else:
            print(f"'{object_to_remove.name}' not found in {list_label} list.")

    def remove_instructor(self, instructor):
        self._remove_object_from_list(instructor, self.instructors, "Instructors")
    def remove_learner(self, learner):
        self._remove_object_from_list(learner, self.learners, "Learners")
    def remove_course(self, course):
        self._remove_object_from_list(course, self.courses, "Courses")        
        
    def get_instructors_for_course(self, course_name):
        return [i.name for i in self.instructors if course_name in i.taught_courses]
    
    def get_courses(self):
        '''Dynamically builds the course list by scanning instructors'''
        courses = {} # course_name -> list of instructor names
        for instructor in self.instructors:
            for course_name in instructor.taught_courses:
                if course_name not in courses:
                    courses[course_name] = []
                courses[course_name].append(instructor.name)
        return courses

    def print_courses(self):
        for course_name, instructors in self.get_courses().items():
            print(f"{course_name} is taught by: {', '.join(instructors)}")   
         
    # 2 Methods that make use of self.enrollments
    def enroll(self, learner, course):
        # Check if already enrolled
        if any(e.learner == learner and e.course == course for e in self.enrollments):
            print(f"{learner.name} is already enrolled in {course.name}.")
            return
        enrollment = Enrollment(learner, course)
        self.enrollments.append(enrollment)
        print(f"{learner.name} successfully enrolled in {course.name}.")

    def drop(self, learner, course):
        for e in self.enrollments:
            if e.learner == learner and e.course == course:
                self.enrollments.remove(e)
                print(f"{learner.name} dropped {course.name}.")
                return
        print(f"{learner.name} is not enrolled in {course.name}.")    

    
    # CLI interface methods
    def validate_email(self, email):
        """
        Validates email format and checks for duplicates in self.learners.
        Returns True if valid, False otherwise.
        """
        # Basic regex for email format
        pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    
        if not re.match(pattern, email):
            print("Invalid email format. Please enter a valid email (e.g., user@example.com).")
            return False
    
        # Check if email already exists
        if any(l.email == email for l in self.learners):
            print("This email is already registered. Please use a different one.")
            return False
    
        return True
        
    def add_learner_from_cli(self):
        '''CLI input with email validation'''
        name = input('Enter learner full name:').strip()

        while True:
            email = input('Enter learner email:').strip()
            if self.validate_email(email):
                break # valid email

        learner = Learner(name, email)
        self.learners.append(learner)
        print(f"Learner '{name}' successfully added")       
           
    def enrollment_via_cli(self):
        learner_name = input("Enter learner name: ").strip()
        course_name = input("Enter course name: ").strip()

        learner = next((l for l in self.learners if l.name == learner_name), None)
        course = next((c for c in self.courses if c.name == course_name), None)

        if not learner:
            print(f"Learner '{learner_name}' not found.")
            return
        if not course:
            print(f"Course '{course_name}' not found.")
            return

        enrollment = Enrollment(learner, course)
        self.enrollments.append(enrollment)

        print(f"Learner '{learner.name}' enrolled in course '{course.name}'.")
        
    def print_learners_for_course(self):
        course_name = input('Enter course name:').strip()
        #print(f"{[l.name for l in self.learners if course_name in l.enrolled_courses]}")
        print(f"{[l.name for l in self.learners if course_name in l.enrolled_courses]}")
    
    def run_cli(self):
        '''CLI interface for interacting with the system.'''        
        print("""STL Tech Backend - Main Menu
        Manage Users Method.
        Options:
            (1) - Add learner
            (2) - Enroll learner in a course
            (3) - Show all courses
            (4) - Show all learners from a course
            (0) - Exit        
        """)
        while True:
            option = int(input('Select option (0 - 4): '))
            match option:
                case 1:
                    self.add_learner_from_cli()
                case 2:
                    self.enrollment_via_cli()
                case 3:
                    self.get_courses() # List dynamically created on the fly from self.instructors. Not stored in backend.courses
                    self.print_courses()
                case 4:
                    self.print_learners_for_course()
                case 0:
                    print("Exit method. Bye!")
                    break
    
    def _print_entity_info(self, name, data_list, entity_label):
        '''Private helper to print an entity from a list, given the name.''' #todo: It might be interesting to also have the possibility to search by ID
        found = next((obj for obj in data_list if obj.name == name), None)
        if found:
            print(found)
        else:
            print(f"{entity_label} '{name}' not found.")

    def print_course_info(self, course_name):
        self._print_entity_info(course_name, self.courses, "Course")

    def print_instructor_info(self, instructor_name):
        self._print_entity_info(instructor_name, self.instructors, "Instructor")    
    

############################################################################################
backend = Backend()
backend.load_instructors_from_excel(file_path)


i1 = Instructor("Daniel Ddd", "dd@mail.com")
i1.add_course("Biology")
i1.add_course("Music")

backend.add_instructor(i1)

i2 = Instructor("Eric Eee", "ee@mail.com")
i2.add_course("Pharmacy")
i3 = Instructor("Francis Fff", "ff@mail.com")
i3.add_course("Pharmacy")

backend.instructors.extend([i2, i3])


print("List of all available instructors:")
for instructor in backend.instructors:
    print(instructor)
    print("-"*30)

print("Instructor names for course Biology:", backend.get_instructors_for_course("Biology"))

print("List of all available courses:")
backend.get_courses() #OBS: list dynamically created on the fly from backend.instructors as inout. Not stored in backend.courses
backend.print_courses()

l1 = Learner("L1", "l1@mail.com")
l1.enroll_course("Algebra")
l2 = Learner("L2", "l2@mail.com")
l2.enroll_course("Algebra")
l3 = Learner("L3", "l3@mail.com")
l3.enroll_course("Algebra")

backend.learners.extend([l1, l2, l3])

backend.run_cli()


Course 'Algebra' added to Alice Aaa.
Course 'Biology' added to Alice Aaa.
Course 'Calculus' added to Alice Aaa.
Course 'Algebra' added to Bob Bbb.
Course 'Calculus' added to Bob Bbb.
Course 'Dentist' added to Bob Bbb.
Course 'Dentist' added to Charles Ccc.
Course 'Pharmacy' added to Charles Ccc.
Course 'Biology' added to Daniel Ddd.
Course 'Music' added to Daniel Ddd.
'Daniel Ddd' successfully added to the Instructors list.
Course 'Pharmacy' added to Eric Eee.
Course 'Pharmacy' added to Francis Fff.
List of all available instructors:
Instructor name: Alice Aaa
Instructor ID: 5535300755
Taught courses: ['Algebra', 'Biology', 'Calculus']
------------------------------
Instructor name: Bob Bbb
Instructor ID: 6623670666
Taught courses: ['Algebra', 'Calculus', 'Dentist']
------------------------------
Instructor name: Charles Ccc
Instructor ID: 618113224
Taught courses: ['Dentist', 'Pharmacy']
------------------------------
Instructor name: Daniel Ddd
Instructor ID: 3458525211
Taught course

Select option (0 - 4):  4
Enter course name: Algebra


['L1', 'L2', 'L3']
